<a href="https://colab.research.google.com/github/chakeaow/EEG-AI/blob/main/code_ees_stress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dataset from Kaggle (Cradits)
Collaborators : prashantgehlot2404  \
you can download dataset from this link : [stress_detection.csv](https://www.kaggle.com/datasets/prashantgehlot2404/eeg-dataset-stress-detection) or this link [Dataset_moreDetail](https://figshare.com/articles/dataset/SAM_40_Dataset_of_40_Subject_EEG_Recordings_to_Monitor_the_Induced-Stress_while_performing_Stroop_Color-Word_Test_Arithmetic_Task_and_Mirror_Image_Recognition_Task/14562090/1?file=27956376) \
Here link is paper of this data where you can see more info : [paper](https://www.sciencedirect.com/science/article/pii/S2352340921010465/pdfft?md5=a06ef90ca4125c8a91555aee4e05cef3&pid=1-s2.0-S2352340921010465-main.pdf)

*Ethical approval was obtained from the Institutional Ethics Committee, Gauhati University,reference no. GUIEC/2019/019 dated: 15/10/2019. The experiment involved human subjects in research whose participation was completely consensual, anonymous, and voluntary. Before opting to partake in the study, the participants were informed about the nature of the study. The data collection was conducted according to the Declaration of Helsinki.*

#### Down load file lable and eeg wave

In [ ]:
!apt-get install -y unrar
!pip install rarfile
!pip install mne

In [ ]:
! wget https://figshare.com/ndownloader/files/27956376

In [ ]:
import rarfile

In [ ]:
rar_path = '27956376'
extract_path = './extracted_files'

with rarfile.RarFile(rar_path) as rf:
    rf.extractall(extract_path)


### perprocess

In [ ]:
import numpy as np
import mne # eeg libery

from scipy.io import loadmat # to load matlab file
import glob
import os


import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler


In [ ]:
## function to pre-prpcess

def Download_file(folder_path,mat_files,i) :
    file_path = os.path.join(folder_path, mat_files[i])
    data = loadmat(file_path)
    sample_data = data.copy() #dic key that contain data is Clean_data
    raw_data = sample_data['Clean_data'] #loading
    return (raw_data)

def create_INFO (raw_data) :

    n_channel, n_time = raw_data.shape #find number of chan , range of f
    #create info : info = mne.create_info(channel_names, sampling_rate, channel_types)

    info = mne.create_info(
    ch_names = chan_list ,
    sfreq = f,
    ch_types = ["eeg"]* n_channel )
    info.set_montage('standard_1020') #add node location


    raw = mne.io.RawArray(raw_data, info) #create data for mne
    #raw.plot(n_channels=12, duration=6, scalings= "auto")
    return(raw)

def Re_reference (raw) :
    raw_reref = raw.copy().set_eeg_reference(ref_channels="average", projection=True)  # projection by average
    #print("projection by average plot :")
    #raw_processed.plot(n_channels=12, duration=6, scalings= "auto")
    #raw_processed.apply_proj()
    return (raw_reref)

def filtered(raw_reref) :
    raw_fillter = raw_reref .copy().filter(l_freq= 0.5 , h_freq= 45)
    #raw_fillter.plot(n_channels=12, duration=6, scalings= "auto")
    #raw_fillter.pick(picks='eeg').compute_psd(fmax=raw_processed.info['sfreq']/2).plot()
    return (raw_fillter)

def ica(raw_fillter):
    ica = mne.preprocessing.ICA(n_components=32, random_state=97, max_iter='auto')
    ica.fit(raw_fillter)
    ica.plot_sources(raw_fillter, show_scrollbars=False)  #EEG wave
    ica.plot_components() #Topographic



Clean Lables

In [ ]:
#downlodd file
file_xls = pd.read_excel('/content/extracted_files/Data/scales.xls')
print(file_xls.head(),"\n\n")


In [ ]:
file_xls.columns

In [ ]:
newColumname = ['Subject No.', "ArithmeticT1", 'MirrorT1', 'StroopT1', "ArithmeticT2","MirrorT2", 'StroopT2', 'ArithmeticT3', 'MirrorT3', 'StroopT3']
file_xls.columns =  newColumname
file_xls.columns

In [ ]:
file_xls = file_xls.drop(0)
file_xls

In [ ]:
scell = {}
sub = file_xls[newColumname[0]]
#colect allevent lables
Ari1,Ari2,Ari3 = file_xls[newColumname[1]], file_xls[newColumname[4]], file_xls[newColumname[7]]
stro1,stro2,stro3 = file_xls[newColumname[3]], file_xls[newColumname[6]], file_xls[newColumname[9]]
mir1,mir2,mir3 = file_xls[newColumname[2]], file_xls[newColumname[5]], file_xls[newColumname[8]]

for i in sub :
    n = int(i)
    score = [Ari1[n],Ari2[n]]
    a = "Arithmetic" + str(n) + 't_'
    scell[a+"1"] = Ari1[n]; scell[a+"2"] = Ari2[n]; scell[a+"3"] = Ari3[n]

    s = "Stroop" + str(n) + 't_'
    scell[s+'1'] = stro1[n]; scell[s+'2'] = stro2[n]; scell[s+'3'] = stro3[n]

    m = "Mirror" + str(n) + 't_'
    scell[m+'1'] = mir1[n]; scell[m+'2'] = mir2[n]; scell[m+'3'] = mir3[n]

    r = "Relax" + str(n) + 't_'
    scell[r+'1'],scell[r+'2'],scell[r+'3'] = 1,1,1

print(scell,"\n",type(scell))

###---------------- convert in to 3 level -----------------# (you can change in to two level here)

def scell_to_level(x):
    if x <= 4 : return -1 #negative
    elif x <= 6: return 0 #normal
    return 1 # positive



In [ ]:
count = [0,0,0,0,0,0,0,0,0,0]
for i in scell.values() :
    count[i-1] += 1

print("Density of rate :",count)
print("Total data :",sum(count))

perprocess data (in actuly it's already preprocess by we will redo in agann for sure)

In [ ]:
# data from paper page 5
chan_list = ['Cz', 'Fz', 'Fp1', 'F7', 'F3', 'FC1', 'C3', 'FC5', 'FT9', 'T7', 'CP5', 'CP1', 'P3', 'P7', 'PO9', 'O1', 'Pz', 'Oz', 'O2', 'PO10', 'P8', 'P4', 'CP2', 'CP6', 'T8', 'FT10', 'FC6', 'C4', 'FC2', 'F4', 'F8', 'Fp2']
f = 128

In [ ]:
folder_path = "/content/extracted_files/Data/filtered_data"
mat_files = [f for f in os.listdir(folder_path) if f.endswith('.mat')] #find all .mat file
print("we have :",len(mat_files),'file')
print(mat_files,"\n\n")

data,event = [],[]
label = []
start= 0
#task of this paper
event_dict = {
     'Relax' : 1,
     'Stroop' : 2,
     'Mirror' : 3,
     'Arithmetic' : 4
     }
filter = ['Arithmetic','Mirror','Stroop', 'Relax']

In [ ]:
for i in range (0,len(mat_files)):
     name = mat_files[i].split("_")
     name.remove("sub")
     if "image" in name : name.remove("image")
     if name[0] not in  filter : #you can play or fillter each task in here
          continue
     event_name = name[0] ; trial = name[-1][-5] ; sub = name[1] #cataaorice each data cy file name
     key = event_name + sub + "t_" + trial
     #print(key)

     ##--------------------- preprocess ----------------------##
     raw_data = Download_file(folder_path,mat_files,i)
     raw = create_INFO(raw_data)
     raw_rr = Re_reference (raw)
     raw_ft = filtered(raw_rr)
     #ica(raw_ft)

     ##--------------------- ephoching event manage ----------------------##
     event_id = event_dict.get(event_name)
     revious_value = 0
     events = np.array([start,revious_value,event_id])
     start += 3200

     event += [events]
     data += [raw_ft]

     ##_______ match lable and task __________#
     if event_id not in [1]: # if you want to analite but dont want to use as ml data filter in here
          label += [ scell_to_level(scell[key]) ]

In [ ]:
event = np.array(event)
print(event)
print(data)
data = mne.concatenate_raws(data)
print(data)

In [ ]:
print(data.info)

In [ ]:
ica(data)

In [ ]:
data_epochs = mne.Epochs(data, event, event_dict, tmin=11, tmax=16, baseline=(11,11.5), event_repeated= 'merge', preload=True)
print(data_epochs.drop_log)

Not setting metadata
480 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 480 events and 641 original time points ...
0 bad epochs dropped
((), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (

In [ ]:
print(data_epochs.info)
print("Over all case")
data_epochs.average().plot_topomap(times=[12, 12.5, 13, 15])
print("In relax task")
relax_data = data_epochs['Relax']
relax_data.average().plot_topomap(times=[12, 12.5, 13, 15])

print("in Stroop task")
stroop_data = data_epochs['Stroop']
stroop_data.average().plot_topomap(times=[12, 12.5, 13, 15])

print("in Mirror task")
mirror_data = data_epochs['Mirror']
mirror_data.average().plot_topomap(times=[12, 12.5, 13, 15])

print("in Arithmetic task")
Arithmetic_data = data_epochs['Arithmetic']
Arithmetic_data.average().plot_topomap(times=[12, 12.5, 13, 15])

In [ ]:
epochs = data_epochs['Stroop','Mirror', 'Arithmetic']
epochs.info

# Analysis parth

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

In [ ]:
#dimentional reduction
data = epochs.get_data()
n_epochs, n_channels, n_times = data.shape
data = data.reshape(n_epochs, n_channels * n_times)

labels = np.array(label)

### split data to training sat and testing set

In [ ]:

from sklearn.model_selection import train_test_split,GridSearchCV

print(data.shape,labels.shape)
#split data to trian and test
train_data,test_data,train_labels,test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

In [ ]:
print(train_data,"\n")

### Synthetic Minority Over-sampling
if you don't wanna use Over-sampling data do not run this cell

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smote = SMOTE(random_state=42)
train_data, train_labels = smote.fit_resample(train_data, train_labels)

In [ ]:
c = [0,0,0]
for i in (train_labels):
    if i == -1 :
        c[0] += 1
    elif i == 0 :
        c[1] += 1
    else :
        c[2] += 1
print("Train data lables : ",c)

c = [0,0,0]
for i in (test_labels):
    if i == -1 :
        c[0] += 1
    elif i == 0 :
        c[1] += 1
    else :
        c[2] += 1
print("Test data lables : ",c)

In [ ]:
from sklearn.metrics import mean_squared_error,confusion_matrix, ConfusionMatrixDisplay,accuracy_score,classification_report

## Train_RandomForest model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

### Hyperparameters junning

In [ ]:
param_grid = {
    'n_estimators': [350, 390, 420],
    'max_depth': [10 ,15, 20],
    'min_samples_split': [5, 11, 15],
    'min_samples_leaf': [1],
    'class_weight': [ 'balanced_subsample']
} # range of paramiter

In [ ]:
rf = GridSearchCV(estimator=RandomForestClassifier(random_state=42), param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

rf.fit(train_data, train_labels)
print("Best Parameters:", rf.best_params_)
print("Best Score:", rf.best_score_)

In [ ]:
best_rf = rf.best_estimator_
rf_pre = best_rf.predict(test_data)

### result
As a result, you can see it's overfitting and failing to identify the negative class, which is my main goal.

In [ ]:
##confusion_matrix

accuracy = accuracy_score(test_labels, rf_pre) #add train
print("Accuracy :",accuracy)

print('\nTest Set: Classification report')
print(classification_report(test_labels, rf_pre))

cm = confusion_matrix(test_labels, rf_pre)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Reds)
plt.title('Confusion Matrix of RandomForest')
plt.show()

## K Neighbors model
I don’t fully understand this model, so maybe you can improve it.


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

### Hyperparameters junning

In [ ]:
param_grid = {
    'n_neighbors': [5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan'],
    'algorithm' : ['ball_tree', 'brute']
}

In [ ]:
knn = GridSearchCV(estimator=KNeighborsClassifier(), param_grid= param_grid, cv=5, n_jobs=-1, verbose=2)

knn.fit(train_data, train_labels)
print("Best Parameters:", knn.best_params_)
print("Best Score:", knn.best_score_)

In [ ]:
best_knn = knn.best_estimator_
print(best_knn)
knn_pre = best_knn.predict(test_data)

### result
As a result, you can see it's behaving strangely, but I’m not sure why. lol

In [ ]:

accuracy = accuracy_score(test_labels, knn_pre)
print("Accuracy :",accuracy)

print('\nTest Set: Classification report')
print(classification_report(test_labels, knn_pre))

cm = confusion_matrix(test_labels,knn_pre)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix of KNN')
plt.show()

 ## Gradient Boosting Classifier  model
I think this is the best model so far. By the way, I didn’t tune the hyperparameters because it takes too long.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gb =  GradientBoostingClassifier(
    n_estimators =  200,
    learning_rate =  0.1,
    max_depth =  10,
    min_samples_split = 2
)
gb.fit(train_data, train_labels)

In [ ]:
gb_pre2 = gb.predict(test_data)

In [ ]:
accuracy = accuracy_score(test_labels, gb_pre2)
print("Accuracy :",accuracy)

print('\nTest Set: Classification report')
print(classification_report(test_labels, gb_pre2))

cm = confusion_matrix(test_labels, gb_pre2)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Purples)
plt.title('Confusion Matrix of Gradient Boosting')
plt.show()

## Best model feature importance

In [ ]:
feature_importances = gb.feature_importances_

In [ ]:
print(test_data.shape)
n_channels = 32
n_time_points = 641  # 20512/32

channel_importances = np.array([feature_importances[i * n_time_points:(i + 1) * n_time_points].sum() for i in range(n_channels)])

In [ ]:
plt.figure(figsize=(10, 8))
plt.barh(chan_list, channel_importances)
plt.xlabel("Feature Importance")
plt.ylabel("EEG Channel")
plt.title("Feature Importance per EEG Channel in GradientBoostingClassifier")
plt.show()

### Top 16 is :

In [ ]:
plt.figure(figsize=(10, 8))
plt.barh(chan_list, channel_importances)
plt.xlabel("Feature Importance")
plt.ylabel("EEG Channel")
plt.title("Feature Importance per EEG Channel in GradientBoostingClassifier")
plt.show()

In [ ]:
d = {}
for i in range(len(channel_importances)):
    d[channel_importances[i]] = chan_list[i]

channel_importances.sort()
top16 = channel_importances[15:]

print("top 16 channels is")
for i in top16:
    print(d[i],end = " ,")
